# Tool Calling with Azure OpenAI: gpt-4o-mini

In [1]:
from openai import AzureOpenAI

In [2]:
client = AzureOpenAI()

In [3]:
import requests,json
def get_current_weather(city:str)->dict:
    """ can be used to get/fetch current weather information for a city name
    """
    api_key = "6a8b0ac166a37e2b7a38e64416b3c3fe"

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode()
    response = json.loads(response)
    output = {"City Name":city,"weather":response["weather"][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}

    return output

In [4]:
get_current_weather("mumbai")

{'City Name': 'mumbai',
 'weather': 'clear sky',
 'temperature': 298.68,
 'unit': 'kelvin'}

### Tool Integration with LLM
- metadata
    - name of tool/function
    - description of the tool/function
    - arguments details - arguments and its description for the tool/function

In [5]:
tools = [{
    "type":"function",
    "function":{
        "name":"get_current_weather",
        "description":"this function can be used to get the current weather information for any given city.",
        "parameters":{
            "type":"object",
            "properties":{"city":{"type":"string","description":"name of city e.g. mumbai, new york"}},
        },
        "required":['city'],
    }}
]

In [6]:
tool_map={"get_current_weather":get_current_weather}

def generate_response(prompt):

    messages = [{"role":"system","content":"you are a helpful assistant"},
                {"role":"user",'content':prompt}]
    
    first_response = client.chat.completions.create(model='gpt4o',
                                                    messages=messages,
                                                    temperature=0.2,
                                                    tools=tools,tool_choice='auto')
    if first_response.choices[0].message.tool_calls:
        # do something
        print("LLM decided to trigger tool call",first_response.choices[0].message.tool_calls)
        tool_calls = first_response.choices[0].message.tool_calls
        messages.append(first_response.choices[0].message)
        for tool in tool_calls:
            tool_name=tool.function.name
            tool_args = json.loads(tool.function.arguments)
            fun_to_ex = tool_map[tool_name]
            tool_output = fun_to_ex(**tool_args)
            messages.append({"tool_call_id":tool.id,"role":"tool",
                            "name":tool_name,"content":json.dumps(tool_output)})
        second_response = client.chat.completions.create(model='gpt4o',messages=messages,
                                                         temperature=0.1)
        return second_response.choices[0].message.content

    else:
        return first_response.choices[0].message.content

In [7]:
generate_response("What is Artificial Intelligence")

'Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. AI systems can perform tasks that typically require human intelligence, such as understanding natural language, recognizing patterns, solving problems, and making decisions.\n\nThere are several key components and concepts associated with AI:\n\n1. **Machine Learning**: A subset of AI that involves the use of algorithms and statistical models to enable machines to improve their performance on a task through experience.\n\n2. **Deep Learning**: A further subset of machine learning that uses neural networks with many layers (deep neural networks) to analyze various factors of data.\n\n3. **Natural Language Processing (NLP)**: The ability of a machine to understand and respond to human language in a way that is both meaningful and useful.\n\n4. **Computer Vision**: The capability of a machine to interpret and make decisions based on visual data from 

In [8]:
generate_response("what is the current weather in delhi today?")

LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_tmy5DdzupeuZRsuIBeAV7PuQ', function=Function(arguments='{"city":"delhi"}', name='get_current_weather'), type='function')]


'The current weather in Delhi today is characterized by broken clouds, with a temperature of approximately 298.48 K (which is about 25.33°C or 77.6°F).'

In [9]:
result = generate_response("what is the current weather in delhi and bangalore today?")
print(result)

LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_QawQxcokVPSYydnEuRADHLE3', function=Function(arguments='{"city": "delhi"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_LkXTHfP1S6uwKF57JXDH0txH', function=Function(arguments='{"city": "bangalore"}', name='get_current_weather'), type='function')]
As of today, the weather is as follows:

- **Delhi**: Broken clouds with a temperature of approximately 25.33°C (298.48 K).
- **Bangalore**: Few clouds with a temperature of approximately 21.37°C (294.52 K). 

If you need more specific details or forecasts, feel free to ask!


In [10]:
result = generate_response("what is the current weather today?")
print(result)

Could you please specify the city for which you would like to know the current weather?


In [11]:
!python -m pip install wikipedia --quiet

In [12]:
import wikipedia

def get_wikipedia_summary(query:str):
    response = wikipedia.summary(query)
    return response

In [13]:
tools = [{
    "type":"function",
    "function":{
        "name":"get_current_weather",
        "description":"this function can be used to get the current weather information for any given city.",
        "parameters":{
            "type":"object",
            "properties":{"city":{"type":"string","description":"name of city e.g. mumbai, new york"}},
        },
        "required":['city'],
    }},
    {
    "type":"function",
    "function":{
        "name":"get_wikipedia_summary",
        "description":"this function can be used to get information about any place, location, person, historical information.",
        "parameters":{
            "type":"object",
            "properties":{"query":{"type":"string","description":"name of any person, event, location, place"}},
        },
        "required":['query'],
    }}
]

In [14]:
tool_map={"get_current_weather":get_current_weather,
          "get_wikipedia_summary":get_wikipedia_summary}

def generate_response(prompt):

    messages = [{"role":"system","content":"you are a helpful assistant"},
                {"role":"user",'content':prompt}]
    
    while True:
        response = client.chat.completions.create(model='gpt4o',
                                                        messages=messages,
                                                        temperature=0.2,
                                                        tools=tools,tool_choice='auto')
        if response.choices[0].message.tool_calls:
            # do something
            print("LLM decided to trigger tool call",response.choices[0].message.tool_calls)
            tool_calls = response.choices[0].message.tool_calls
            messages.append(response.choices[0].message)
            for tool in tool_calls:
                tool_name=tool.function.name
                tool_args = json.loads(tool.function.arguments)
                fun_to_ex = tool_map[tool_name]
                tool_output = fun_to_ex(**tool_args)
                messages.append({"tool_call_id":tool.id,"role":"tool",
                                "name":tool_name,"content":json.dumps(tool_output)})
        else:
            break
    return response.choices[0].message.content

In [15]:
result = generate_response("what is the current weather in mumbai?")
print(result)

LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_Hb2Hf9bjgLfJgtmWH8GXQA9m', function=Function(arguments='{"city":"mumbai"}', name='get_current_weather'), type='function')]
The current weather in Mumbai is clear sky with a temperature of approximately 298.68 K (which is about 25.5 °C).


In [16]:
result = generate_response("what is the current weather in mumbai and delhi?")
print(result)

LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_Yh6eDcD3EpwUDCNCXMaXh3CX', function=Function(arguments='{"city": "mumbai"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_xbt9wqv9yPEQ0RrnBn9xHJX0', function=Function(arguments='{"city": "delhi"}', name='get_current_weather'), type='function')]
The current weather is as follows:

- **Mumbai**: 
  - Weather: Clear sky
  - Temperature: 298.68 K

- **Delhi**: 
  - Weather: Broken clouds
  - Temperature: 298.48 K

If you need the temperatures in Celsius or Fahrenheit, let me know!


In [17]:
result = generate_response("tell me something about Mumbai and its current weather")
print(result)

LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_v57W5dBG1RAD6xVi9Zno9AT0', function=Function(arguments='{"query": "Mumbai"}', name='get_wikipedia_summary'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_BIhzTpfXbk20s9ORYRjUMWyN', function=Function(arguments='{"city": "Mumbai"}', name='get_current_weather'), type='function')]
### About Mumbai
Mumbai, formerly known as Bombay, is the capital city of the Indian state of Maharashtra. It is the financial capital and the most populous city in India, with an estimated population of 12.5 million. The city is part of the Mumbai Metropolitan Region, which has a population exceeding 23 million, making it one of the most populous metropolitan areas in the world.

Mumbai is located on the Konkan coast and has a deep natural harbor. Historically, it was made up of seven islands that were home to the Koli people. Over the centuries, these islands came under various rulers, including the Portuguese a

In [18]:
result = generate_response("which is the larget city in United Arab Emirates and then tell me what is the weather there?")
print(result)

LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_CKnoOlCh1veXkF8Iuq1jwF6k', function=Function(arguments='{"query":"largest city in United Arab Emirates"}', name='get_wikipedia_summary'), type='function')]
LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_0JQpcJ1oQvZIGb9BrzdE5uyy', function=Function(arguments='{"query":"Dubai"}', name='get_wikipedia_summary'), type='function')]
LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_xXy8NT0ouoBTu8Bu0PNdooLm', function=Function(arguments='{"query":"Dubai"}', name='get_wikipedia_summary'), type='function')]
LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_Sz5YokLN7UWRybGSTKZEAvC4', function=Function(arguments='{"city":"Dubai"}', name='get_current_weather'), type='function')]
The largest city in the United Arab Emirates is **Dubai**. 

Currently, the weather in Dubai is as follows:
- **Condition**: Clear sky
- **Temperatu